In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#读取工作簿和工作簿中的工作表
data_frame=pd.read_excel('Problem_C_Data_Wordle.xlsx')
# print(data_frame)
data = list(reversed(data_frame.loc[ 1: ,"Unnamed: 3"].tolist()))
data_1=list(reversed(data_frame.loc[ : ,"Percent in "]))[:-1]
data_2=list(reversed(data_frame.loc[ 1: ,"Unnamed: 7"].tolist()))
data_3=list(reversed(data_frame.loc[ 1: ,"Unnamed: 8"].tolist()))
data_4=list(reversed(data_frame.loc[ 1: ,"Unnamed: 9"].tolist()))
data_5=list(reversed(data_frame.loc[ 1: ,"Unnamed: 10"].tolist()))
data_6=list(reversed(data_frame.loc[ 1: ,"Unnamed: 11"].tolist()))
data_7=list(reversed(data_frame.loc[ 1: ,"Unnamed: 12"].tolist()))

In [2]:
print(data_1)
print(data_2)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 

In [3]:
# print(data)
letter2idx={}
train_X=[]
train_Y=[]
cnt=0
for item in data:
    letters=list(item)
    x=[0 for i in range(26)]
    y=[data_1[cnt]/100,data_2[cnt]/100,data_3[cnt]/100,data_4[cnt]/100,data_5[cnt]/100,data_6[cnt]/100,data_7[cnt]/100]
    cnt+=1
    for letter in letters:
        if(letter==" "):
            continue
        if(letter not in letter2idx.keys()):
            letter2idx[letter]=len(letter2idx)
            #print(letter,letters)
        #print(letter2idx[letter])
        x[letter2idx[letter]]+=1
    train_X.append(x)
    train_Y.append(y)
#print(train_X,letter2idx)


In [4]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        # self.embedding = nn.Embedding(26, 64)
        self.softmax=nn.Softmax(dim=1)
        self.fc1 = nn.Linear(26, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.3)
        self.relu=nn.ReLU(True)
        self.fc3 = nn.Linear(128, 7)



    def forward(self, x):
    	#x:[batchsize, max_length]
        #out = self.embedding(x)# [batchsize, max_length, embedding_size]
        #这里相当于增加了一个in_channels维度，TextCNN输入的通道维数为1
        #out = out.unsqueeze(1)# [batchsize, 1, max_length, embedding_size]
        # 这里分别使用不同size的卷积核进行卷积，以(2,, 300)为例
        # 输入：[64, 1, 32, 300]
        # 进行卷积得到输出：[64, 128, 32-2+1=31, 1]
        # 经过卷积之后经过一个relu，然后变形成[64, 128, 31]
        # 然后经过1维最大池化得[64, 128, 1]，再变形为[64, 128]
        # 最终将不同卷积核卷积后的结果拼接为,[64, 128*3]
        out=self.fc1(x)
        out=self.fc2(out)
        out=self.relu(out)
        # out = self.dropout(out)
        out = self.fc3(out)
        pred=self.softmax(out)
        return out,pred


In [5]:
batch_size=32
model=TextCNN()
loss_func=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-2)
epochs=500

In [6]:
for k in range(epochs):
    preds=[]
    labels=[]
    for j in range(int(len(train_X)/batch_size)):
        batch=[]
        label=[]
        for i in range(batch_size):
            batch.append(train_X[j*batch_size+i])
            label.append(train_Y[j*batch_size+i])
    
        batch=torch.tensor(batch).to(torch.float)
        label=torch.tensor(label).to(torch.float)
        out,pred=model(batch)
        loss=loss_func(pred,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        preds+=pred.tolist()
        labels+=label.tolist()
    preds=torch.tensor(preds).to(torch.float)
    labels=torch.tensor(labels).to(torch.float)
    loss=loss_func(preds,labels)
    print('Epoch: {:4}, Loss: {:.5f}'.format(k, loss.item()))
    

Epoch:    0, Loss: 0.00478
Epoch:    1, Loss: 0.00299
Epoch:    2, Loss: 0.00247
Epoch:    3, Loss: 0.00223
Epoch:    4, Loss: 0.00220
Epoch:    5, Loss: 0.00194
Epoch:    6, Loss: 0.00203
Epoch:    7, Loss: 0.00202
Epoch:    8, Loss: 0.00173
Epoch:    9, Loss: 0.00133
Epoch:   10, Loss: 0.00128
Epoch:   11, Loss: 0.00118
Epoch:   12, Loss: 0.00119
Epoch:   13, Loss: 0.00126
Epoch:   14, Loss: 0.00114
Epoch:   15, Loss: 0.00113
Epoch:   16, Loss: 0.00123
Epoch:   17, Loss: 0.00117
Epoch:   18, Loss: 0.00127
Epoch:   19, Loss: 0.00140
Epoch:   20, Loss: 0.00122
Epoch:   21, Loss: 0.00128
Epoch:   22, Loss: 0.00108
Epoch:   23, Loss: 0.00081
Epoch:   24, Loss: 0.00083
Epoch:   25, Loss: 0.00084
Epoch:   26, Loss: 0.00113
Epoch:   27, Loss: 0.00139
Epoch:   28, Loss: 0.00159
Epoch:   29, Loss: 0.00171
Epoch:   30, Loss: 0.00153
Epoch:   31, Loss: 0.00159
Epoch:   32, Loss: 0.00170
Epoch:   33, Loss: 0.00158
Epoch:   34, Loss: 0.00121
Epoch:   35, Loss: 0.00117
Epoch:   36, Loss: 0.00106
E

In [7]:
model.eval()
test='ionic'
test=list(test)
test_batch=[[0 for i in range(26)]]
for i in test:
    test_batch[0][letter2idx[i]]+=1
test_batch=torch.tensor(test_batch).to(torch.float)
print(test_batch)

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 2., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])


In [8]:
out,pred=model(test_batch)
print(pred)
mean=0
cnt=1
for weight in pred[0]:
    mean+=weight*cnt
    cnt+=1
print(mean)

tensor([[6.1667e-06, 2.0177e-02, 1.3588e-01, 2.4982e-01, 2.5273e-01, 2.2360e-01,
         1.1779e-01]], grad_fn=<SoftmaxBackward0>)
tensor(4.8770, grad_fn=<AddBackward0>)


In [10]:
data_chi = data_frame.loc[1:60 ,"Unnamed: 3"].tolist()
prediction={}
print(data_chi)
for word in data_chi:
    test=word
    test=list(test)
    test_batch=[[0 for i in range(26)]]
    for i in test:
        test_batch[0][letter2idx[i]]+=1
    test_batch=torch.tensor(test_batch).to(torch.float)
    out,pred=model(test_batch)
    prediction[word]=pred[0].tolist()
    print(test_batch)

['manly', 'molar', 'havoc', 'impel', 'condo', 'judge', 'extra', 'poise', 'aorta', 'excel', 'lunar', 'third', 'slate', 'taper', 'chord', 'probe', 'rival', 'usual', 'spoke', 'apply', 'naive', 'knock', 'braid', 'infer', 'joust', 'amber', 'woken', 'adore', 'torso', 'chafe', 'eject', 'study', 'undue', 'tepid', 'happy', 'clen', 'itchy', 'feast', 'drive', 'prime', 'axiom', 'brave', 'avert', 'glyph', 'there', 'baker', 'snarl', 'maple', 'inane', 'valet', 'medal', 'unite', 'rainy', 'spell', 'begin', 'stale', 'dream', 'photo', 'aloud', 'inept']
tensor([[0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       

In [11]:
print(prediction)

{'manly': [0.0009690403821878135, 0.030068805441260338, 0.17544540762901306, 0.3502339720726013, 0.28928643465042114, 0.13368792831897736, 0.020308438688516617], 'molar': [0.004211779683828354, 0.06830929219722748, 0.2673470973968506, 0.34428614377975464, 0.21597768366336823, 0.08480572700500488, 0.015062268823385239], 'havoc': [0.001061141025274992, 0.017458807677030563, 0.09508875757455826, 0.24099870026111603, 0.31522461771965027, 0.2518423795700073, 0.07832560688257217], 'impel': [0.004535948392003775, 0.06155858188867569, 0.2478121966123581, 0.3473207354545593, 0.2285117208957672, 0.09384476393461227, 0.016416020691394806], 'condo': [0.0017141696298494935, 0.02231675758957863, 0.16596682369709015, 0.317193865776062, 0.2947966754436493, 0.16597266495227814, 0.032039087265729904], 'judge': [0.00029112905031070113, 0.02171826735138893, 0.11826103925704956, 0.31340786814689636, 0.33166515827178955, 0.18289262056350708, 0.031763955950737], 'extra': [0.0004081875376868993, 0.07360334694